In [1]:
# import csv

# csv_file_name = 'scrape_data.csv'

# columns = ["Title", "Authors", "Publish Date", "Abstract", "Keywords", "PDF Name"]
# with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.DictWriter(file, fieldnames=columns)
#     writer.writeheader()

# def add_row_to_csv(data):
#     with open(csv_file_name, mode='a', newline='', encoding='utf-8') as file:
#         writer = csv.DictWriter(file, fieldnames=columns)
#         writer.writerow(data)


In [2]:
import csv

csv_file_name = 'scrape_data.csv'

columns = ["Title", "Authors", "Publish Date", "Abstract", "Keywords", "PDF Name"]

def add_row_to_csv(data):
    with open(csv_file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=columns)
        writer.writerow(data)

In [3]:
def get_credentials():
    with open('credentials.txt', 'r') as file:
        lines = file.readlines()
        username = lines[0].strip()
        password = lines[1].strip()
        file.close()
        return username, password

In [4]:
def overwrite_line(line_number, new_value):
    with open('progress.txt', 'r') as file:
        lines = file.readlines()
        
    if 0 <= line_number < len(lines):
        lines[line_number] = f"{new_value}\n" 
        with open('progress.txt', 'w') as file:
            file.writelines(lines)
        return True
    else:
        return False

def read_numbers():
    with open('progress.txt', 'r') as file:
        numbers = [int(line.strip()) for line in file]
    return numbers[0], numbers[1], numbers[2]


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
import requests
import time
import urllib
import os
import shutil
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Initial_path = r"C:\Users\yitong\Desktop\Research-Assistant\Scrape\PDF"

def get_info(wd, year, issue, n, article_button):
    ActionChains(wd).move_to_element(article_button).key_down(Keys.CONTROL).click(article_button).key_up(Keys.COMMAND).perform()
    time.sleep(5)
    window_handles = wd.window_handles
    wd.switch_to.window(window_handles[1])
    time.sleep(5)
    try:
        dissmiss_button = wd.find_element(By.CLASS_NAME, '_pendo-close-guide')
        dissmiss_button.click()
    except:
        pass
    
    try:
        # print("No expand button")
        # 2009 and previous
        time.sleep(5)
        element = wd.find_element(By.CLASS_NAME, 'tool-option')
        element.click()
        time.sleep(5)
        button = wd.find_element(By.PARTIAL_LINK_TEXT, "关闭")
        button.click()
        time.sleep(5)
        link = wd.find_element(By.PARTIAL_LINK_TEXT, "索引")
        link.click()
        time.sleep(5)

        window_handles = wd.window_handles
        wd.switch_to.window(window_handles[2])

        try:
            expand_button = wd.find_element(By.CSS_SELECTOR, '[id^="MoreLink_abstractSummary_MSTAR_"]')
            expand_button.click()
        except:
            pass

        try:
            abstract = wd.find_element(By.CLASS_NAME, 'abstract').text
        except NoSuchElementException:
            abstract = None

        
        xpath = "//div[contains(text(), '标题')]/following-sibling::div"
        headline_element = wd.find_element("xpath", xpath)
        title = headline_element.text
        
        xpath = "//div[contains(text(), '作者')]/following-sibling::div"
        headline_element = wd.find_element("xpath", xpath)
        authors = headline_element.text

        xpath = "//div[contains(text(), '出版日期')]/following-sibling::div"
        headline_element = wd.find_element("xpath", xpath)
        publish_date = headline_element.text

        xpath = "//div[contains(text(), '主题')]/following-sibling::div"
        headline_element = wd.find_element("xpath", xpath)
        keywords = headline_element.text
        keywords_list = keywords.split()  # Splits the string into a list by spaces
        keywords_string = ", ".join(keywords_list)  # Joins the list into a string separated by commas
        keyword_list = [keyword.strip().replace(",", "") for keyword in keywords.split(";") if keyword.strip()]
        # Join the cleaned list back into a string with commas
        keywords = ", ".join(keyword_list)


        xpath = "//div[contains(text(), 'ProQuest 文档 ID')]/following-sibling::div"
        headline_element = wd.find_element("xpath", xpath)
        ProQuest_id = headline_element.text

        pdf_file_name = ProQuest_id

        time.sleep(30)
        filename = max([Initial_path + "\\" + f for f in os.listdir(Initial_path)], key=os.path.getctime)
        shutil.move(filename, os.path.join(Initial_path, pdf_file_name + ".pdf"))

        wd.close()
        wd.switch_to.window(window_handles[1])
        wd.close()
        wd.switch_to.window(window_handles[0])
        

        return title, authors, publish_date, abstract, keywords, pdf_file_name

    except Exception as e:
        print(e)
        try:
            expand_button = wd.find_element(By.CSS_SELECTOR, '[id^="MoreLink_abstractSummary_MSTAR_"]')
            expand_button.click()
            print(3)
        except:
            pass
        
    try:
        abstract = wd.find_element(By.CLASS_NAME, 'abstract').text
    except NoSuchElementException:
        abstract = None

    xpath = "//div[contains(text(), '标题')]/following-sibling::div"
    headline_element = wd.find_element("xpath", xpath)
    title = headline_element.text
    
    xpath = "//div[contains(text(), '作者')]/following-sibling::div"
    headline_element = wd.find_element("xpath", xpath)
    authors = headline_element.text

    xpath = "//div[contains(text(), '出版日期')]/following-sibling::div"
    headline_element = wd.find_element("xpath", xpath)
    publish_date = headline_element.text

    xpath = "//div[contains(text(), '主题')]/following-sibling::div"
    headline_element = wd.find_element("xpath", xpath)
    keywords = headline_element.text
    keywords_list = keywords.split()  # Splits the string into a list by spaces
    keywords_string = ", ".join(keywords_list)  # Joins the list into a string separated by commas
    keyword_list = [keyword.strip().replace(",", "") for keyword in keywords.split(";") if keyword.strip()]
    # Join the cleaned list back into a string with commas
    keywords = ", ".join(keyword_list)


    xpath = "//div[contains(text(), 'ProQuest 文档 ID')]/following-sibling::div"
    headline_element = wd.find_element("xpath", xpath)
    ProQuest_id = headline_element.text

    pdf_file_name = ProQuest_id

    try:
        link_element = wd.find_element(By.CLASS_NAME, 'unpaywallFulltextButton')
        link_element.click()
        time.sleep(5)
        window_handles = wd.window_handles
        time.sleep(3)
        wd.switch_to.window(window_handles[1])
        time.sleep(3)
        wd.close()
        time.sleep(3)
        wd.switch_to.window(window_handles[0])

    except NoSuchElementException:

        image_element = wd.find_element("xpath", '//img[@alt="Find it! @ NUS Lib"]')
        image_element.click()
        window_handles = wd.window_handles
        wd.switch_to.window(window_handles[2])
        time.sleep(3)
        link_element = wd.find_element("link text", 'Click to get Full Text')
        time.sleep(1)
        link_element.click()
        window_handles = wd.window_handles
        wd.switch_to.window(window_handles[3])
        time.sleep(3)
        pdf_links = wd.find_elements(By.CSS_SELECTOR, "a.btn[href*='/doi/epdf/']")
        for link in pdf_links:
            url = link.get_attribute('href')
            wd.get(url)
            break

        try:
            time.sleep(60)
            download_icon = wd.find_element(By.XPATH, "//span[@class='icon material-icons' and text()='get_app']")
            download_icon.click()
            time.sleep(5)
            wd.close()
            wd.switch_to.window(window_handles[2])
            wd.close()
            wd.switch_to.window(window_handles[1])
            wd.close()
            wd.switch_to.window(window_handles[0])

            time.sleep(30)
            filename = max([Initial_path + "\\" + f for f in os.listdir(Initial_path)], key=os.path.getctime)
            shutil.move(filename, os.path.join(Initial_path, pdf_file_name + ".pdf"))

        except:
            # informs pubs online page
            # need to locate the h5 class
            try:
                element = wd.find_element(By.XPATH, "//h5[@class='hlFld-Title meta__title meta__title__margin']/a")
                element.click()
                time.sleep(5)
                window_handles = wd.window_handles
                wd.switch_to.window(window_handles[4])
                time.sleep(3)
                pdf_links = wd.find_elements(By.CSS_SELECTOR, "a.btn[href*='/doi/epdf/']")
                for link in pdf_links:
                    url = link.get_attribute('href')
                    wd.get(url)
                    break
                time.sleep(60)
                download_icon = wd.find_element(By.XPATH, "//span[@class='icon material-icons' and text()='get_app']")
                download_icon.click()
                time.sleep(5)
                wd.close()
                wd.switch_to.window(window_handles[3])
                wd.close()
                wd.switch_to.window(window_handles[2])
                wd.close()
                wd.switch_to.window(window_handles[1])
                wd.close()
                wd.switch_to.window(window_handles[0])
                time.sleep(30)
                filename = max([Initial_path + "\\" + f for f in os.listdir(Initial_path)], key=os.path.getctime)
                shutil.move(filename, os.path.join(Initial_path, pdf_file_name + ".pdf"))

            except:
                with open('error.txt', 'a') as file:
                    file.write(f"{year}, {issue}, {n}, {title}\n")
                    file.close()
                    time.sleep(5)
                    all_windows = wd.window_handles
                    for window in all_windows[1:]:  
                        wd.switch_to.window(window) 
                        wd.close() 
                    wd.switch_to.window(all_windows[0])

    return title, authors, publish_date, abstract, keywords, pdf_file_name


# download -> rename -> write to csv

In [6]:
def get_from_one_booklet(wd, year, issue, article_index=0):
    articles = wd.find_elements(By.CLASS_NAME, 'truncatedResultsTitle')
    for n in range(article_index, len(articles)):
        overwrite_line(2, n)
        article = articles[n]
        print("article_index")
        print(n)
        title, authors, publish_date, abstract, keywords, pdf_file_name = get_info(wd, year, issue, n, article)
        add_row_to_csv({
            "Title": title,
            "Authors": authors,
            "Publish Date": publish_date,
            "Abstract": abstract,
            "Keywords": keywords,
            "PDF Name": pdf_file_name
        })
    overwrite_line(2, 0)

In [7]:
# 先处理这个年份这个issue剩下的article
# 再处理这个年份剩下的issue
# 再处理剩下的年份

def recover_from_breakpoint(year, issue, article_index):
    
    user_id, password = get_credentials()

    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
        "download.default_directory": Initial_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True 
    })

    wd = webdriver.Chrome(service = Service(r'chromedriver.exe'), options=options)
    wait = WebDriverWait(wd, 60)
    
    wd.get("http://libproxy1.nus.edu.sg/login?url=https://www.proquest.com/publication/29794?accountid=13876&decadeSelected=2020+-+2029&yearSelected=2023&monthSelected=09&issueNameSelected=02023Y09Y01$23Sep+2023$3b++Vol.+34+$283$29")

    button = wd.find_element(By.ID, 'btn_nus')
    button.click()
    wait.until(EC.element_to_be_clickable((By.ID, 'userNameInput')))
    user_id_box = wd.find_element(By.ID, 'userNameInput')
    password_box = wd.find_element(By.ID, 'passwordInput')
    user_id_box.send_keys(user_id)
    password_box.send_keys(password)
    submit_button = wd.find_element(By.ID, 'submitButton')
    submit_button.click()
    wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler')))
    cookie_button = wd.find_element(By.ID, 'onetrust-accept-btn-handler')
    time.sleep(2)
    cookie_button.click()

    dropdown_element_year = wait.until(EC.presence_of_element_located((By.ID, 'yearSelected')))
    button_locator = "input[id^='searchPubIssues_']"

    # 先处理这个年份这个issue剩下的article
    print("先处理这个年份这个issue剩下的article")
    option_index = year
    overwrite_line(0, option_index)
    print(">>>>>>year_index")
    print(option_index)
    time.sleep(2)
    dropdown_year = Select(wait.until(EC.element_to_be_clickable((By.ID, 'yearSelected'))))
    time.sleep(2)
    option = dropdown_year.options[option_index]  # Get current option by index
    time.sleep(2)
    dropdown_year.select_by_visible_text(option.text)
    time.sleep(2)
    # wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, button_locator))).click()
    # time.sleep(2)
    dropdown_element_issue = wait.until(EC.presence_of_element_located((By.ID, 'issueSelected')))
    time.sleep(3)
    dropdown_issue_options = Select(dropdown_element_issue).options
    time.sleep(2)
    optionn_index = issue
    overwrite_line(1, optionn_index)
    print(">>>issue_index")
    print(optionn_index)
    time.sleep(2)
    dropdown_issue = Select(wait.until(EC.element_to_be_clickable((By.ID, 'issueSelected'))))
    time.sleep(2)
    optionn = dropdown_issue.options[optionn_index]  # Get current option by index
    time.sleep(2)
    dropdown_issue.select_by_visible_text(optionn.text)
    time.sleep(2)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, button_locator))).click()
    time.sleep(2)
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'truncatedResultsTitle')))
    time.sleep(2)
    get_from_one_booklet(wd, option_index, optionn_index, article_index)
    time.sleep(2)
    dropdown_element_year = wait.until(EC.presence_of_element_located((By.ID, 'yearSelected')))
    overwrite_line(2, 0)

    # 再处理这个年份剩下的issue
    print("再处理这个年份剩下的issue")
    option_index = year
    overwrite_line(0, option_index)
    overwrite_line(2, 0) # set article_index to 0
    print(">>>>>>year_index")
    print(option_index)
    time.sleep(2)
    dropdown_year = Select(wait.until(EC.element_to_be_clickable((By.ID, 'yearSelected'))))
    time.sleep(2)
    option = dropdown_year.options[option_index]  # Get current option by index
    time.sleep(2)
    dropdown_year.select_by_visible_text(option.text)
    time.sleep(2)
    # wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, button_locator))).click()
    # time.sleep(2)
    dropdown_element_issue = wait.until(EC.presence_of_element_located((By.ID, 'issueSelected')))
    time.sleep(3)
    dropdown_issue_options = Select(dropdown_element_issue).options
    time.sleep(2)
    for optionn_index in range(issue+1, len(dropdown_issue_options)):
        print(">>>issue_index")
        print(optionn_index)
        overwrite_line(1, optionn_index)
        overwrite_line(2, 0) # set article_index to 0
        time.sleep(2)
        article_index = 0
        dropdown_issue = Select(wait.until(EC.element_to_be_clickable((By.ID, 'issueSelected'))))
        time.sleep(2)
        optionn = dropdown_issue.options[optionn_index]  # Get current option by index
        time.sleep(2)
        dropdown_issue.select_by_visible_text(optionn.text)
        time.sleep(2)
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, button_locator))).click()
        time.sleep(2)
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'truncatedResultsTitle')))
        time.sleep(2)
        get_from_one_booklet(wd, option_index, optionn_index, article_index)
        time.sleep(2)
        dropdown_element_year = wait.until(EC.presence_of_element_located((By.ID, 'yearSelected')))
        overwrite_line(2, 0)
    overwrite_line(2, 0)
    overwrite_line(1, 0)


    # 再处理剩下的年份
    print("再处理剩下的年份")
    option_index = year+1
    #while option_index < len(Select(dropdown_element_year).options):
    while option_index < 22:
        print(">>>>>year_index")
        print(option_index)
        overwrite_line(0, option_index)
        overwrite_line(2, 0) # set article_index to 0
        time.sleep(2)
        dropdown_year = Select(wait.until(EC.element_to_be_clickable((By.ID, 'yearSelected'))))
        time.sleep(2)
        option = dropdown_year.options[option_index]  # Get current option by index
        time.sleep(2)
        dropdown_year.select_by_visible_text(option.text)
        time.sleep(2)
        # wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, button_locator))).click()
        # time.sleep(2)
        dropdown_element_issue = wait.until(EC.presence_of_element_located((By.ID, 'issueSelected')))
        time.sleep(2)
        dropdown_issue_options = Select(dropdown_element_issue).options
        time.sleep(2)
        for optionn_index in range(len(dropdown_issue_options)):
            print(">>>issue_index")
            print(optionn_index)
            overwrite_line(2, 0) # set article_index to 0
            overwrite_line(1, optionn_index)
            time.sleep(2)
            article_index = 0
            dropdown_issue = Select(wait.until(EC.element_to_be_clickable((By.ID, 'issueSelected'))))
            time.sleep(2)
            optionn = dropdown_issue.options[optionn_index]  # Get current option by index
            time.sleep(2)
            dropdown_issue.select_by_visible_text(optionn.text)
            time.sleep(2)
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, button_locator))).click()
            time.sleep(2)
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'truncatedResultsTitle')))
            time.sleep(2)
            get_from_one_booklet(wd, option_index, optionn_index, article_index)
            time.sleep(2)
            dropdown_element_year = wait.until(EC.presence_of_element_located((By.ID, 'yearSelected')))
            overwrite_line(2, 0)
        overwrite_line(2, 0)
        overwrite_line(1, 0)
        option_index += 1
        overwrite_line(0, option_index)

In [8]:
while True:
    try:
        year, issue, article_index = read_numbers()
        if year == 22:
            break
        recover_from_breakpoint(year, issue, article_index)
    except Exception as e:
        print(e)
        print("Failed")

先处理这个年份这个issue剩下的article
>>>>>>year_index
11
>>>issue_index
2
article_index
0
Message: no such element: Unable to locate element: {"method":"partial link text","selector":"关闭"}
  (Session info: chrome=121.0.6167.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E5FF5E42+3538674]
	(No symbol) [0x00007FF7E5C14C02]
	(No symbol) [0x00007FF7E5AC5AEB]
	(No symbol) [0x00007FF7E5B0BF4E]
	(No symbol) [0x00007FF7E5B0C0CC]
	(No symbol) [0x00007FF7E5B4E477]
	(No symbol) [0x00007FF7E5B2F0EF]
	(No symbol) [0x00007FF7E5B4BDE3]
	(No symbol) [0x00007FF7E5B2EE53]
	(No symbol) [0x00007FF7E5AFF514]
	(No symbol) [0x00007FF7E5B00631]
	GetHandleVerifier [0x00007FF7E6026CAD+3738973]
	GetHandleVerifier [0x00007FF7E607C506+4089270]
	GetHandleVerifier [0x00007FF7E6074823+4057299]
	GetHandleVerifier [0x00007FF7E5D45C49+720121]
	(No symbol) [0x00007FF7E5C2126F]
	(No symbol)